# Código para pegar Letras na Internet Usando Beautiful Soup e DOM

## Já há um arquivo com as letras em XML chamado 'Lyrics'

In [1]:
#Importa todas as bibliotecas necessárias.
import requests
from bs4 import BeautifulSoup as bs
from lxml import etree, objectify
from io import BytesIO
import time

In [2]:
def create_xml(site,ident):
    #Encontra o elemento no site que contém a letra.
    letra = site.find("div", attrs={"class":"cnt-letra p402_premium"})

    #Encontra o título (title), o artista (artist) e o estilo (style).
    header = soup.find("div", attrs={"class":"cnt-head_title"})
    title = header.find("h1")
    artist = header.find('span')
    aux_style_1 = soup.find("div", attrs={"id":"breadcrumb","class":"breadcrumb cor_1 g-1"})
    aux_style_2 = aux_style_1.find_all("span",attrs={"itemprop":"name"})
    style = aux_style_2[1]
    
    #Cria o elemento MUSICA e nele insere os elementos titulo, artista e estilo.
    appt = objectify.Element("MUSICA")
    appt.set("id", str(ident))
    appt.set("titulo", title.text)
    appt.set("artista", artist.text)
    appt.set("estilo", style.text)
    
    #Faz de cada estrofe da letra um elemento em MUSICA chamado ESTROFE.
    estrofes = letra.find_all("p")
    for p in estrofes:
        stanza = etree.SubElement(appt,"ESTROFE")
        for br in p.find_all("br"):
            br.replace_with(r"////")
        estrofe = p.text
        # Faz de cada verso da estrofe um elemento em ESTROFE chamado VERSO.
        verses = estrofe.split('////')
        if '' in verses: verses.remove('')
        for v in verses:
            verse = etree.SubElement(stanza,"VERSO")
            verse._setText(v)    
    
    #Adiciona o elemento MUSICA ao root.
    root.append(appt)
    
    # Remove a anotações não relevantes.
    objectify.deannotate(root)
    etree.cleanup_namespaces(root)
    
    # Cria a string em formato xml.
    parser = etree.XMLParser(remove_blank_text=True)
    file_obj = BytesIO(etree.tostring(root))
    tree = etree.parse(file_obj, parser)
    
    #Cria ou atualiza o arquivo .xml
    try:
        with open("Lyrics.xml", "wb") as xml_writer:
            tree.write(xml_writer, pretty_print=True)
    except IOError:
        pass

In [3]:
# Lista de estilos e músicas que já foram obtidas.
lista_est = []
lista_ref = []

In [4]:
#Inicia a contagem do tempo de execução.
start_time = time.time()

#Recebe o site com a lista de estilos musicais.
r = requests.get("https://www.letras.mus.br/estilos/")
soup = bs(r.text)

#Cria a tag do root.
xml = '''<!--?xml version="1.0" encoding="UTF-8"?-->
<Letras>
</Letras>
'''
#Cria o root.
root = objectify.fromstring(xml)

#Entra no site com os estilos musicais.
lista_estilos = soup.find("ul", attrs = {"class":"cnt-list cnt-list--col2"})
estilos = lista_estilos.find_all('li')
# A variável 'i' é o identificador (ID) de cada música no arquivo xml.
i = 1

# A variável 'quant' diz quantas músicas de cada estilo será buscada.
# Se for omitida a variável 'quant', será obtido até 1000 músicas de cada estilo.
quant = 200

# Realiza um loop pelos estilos musicais.
for e in estilos:
    if e.text == 'Clássico':
        break
    start = time.time()
    numero = 0
    # Entra no site de determinado estilo.
    a = e.find('a') ; a = a['href']
    
    # Essa parte do site deve estar em construção, não há músicas, por isso pulei ela.
    # Até porque Post Rock é um estilo musical que não costuma ter letras.
    if a == '/estilos/post-rock/': continue
    
    # Imprime o estilo musical que está sendo buscado.
    # Começa em 'alternativo' e termina em 'zouk'
    print(a[9:-1])
    
    a = a.replace('estilos','mais-acessadas')
    r = requests.get("https://www.letras.mus.br"+ a)
    soup = bs(r.text)
    # Realiza um loop com as músicas do estilo.
    lista_musicas = soup.find("ol", attrs = {"class":"top-list_mus cnt-list--col1-3"})
    musica = lista_musicas.find_all("li")
    for li in musica:
        link_musica = li.find("a")
        
        #Checa se a música já foi registrada ou não.
        if link_musica["href"] not in lista_ref:
            
            #Pega a letra da música e insere ela no xml com o 'root'.
            lista_ref.append(link_musica["href"])
            r = requests.get("https://www.letras.mus.br" + link_musica["href"])
            soup = bs(r.text) ; create_xml(soup,i) ; i += 1
            
            # Checa se passou do limite 'quant' de músicas do estilo.
            numero += 1
            if numero >= quant: break
    if e not in lista_est: lista_est.append(e)
    else: print(a[16:-1],'já foi obtido.')
    print('Tempo',"--- %s seconds ---" % (time.time() - start))
#Imprime o tempo de execução
print('Tempo total',"--- %s seconds ---" % (time.time() - start_time))

alternativo
Tempo --- 21.720256805419922 seconds ---
axe
Tempo --- 28.644578456878662 seconds ---
blues
Tempo --- 38.84819412231445 seconds ---
bolero
Tempo --- 40.302024841308594 seconds ---
bossa-nova
Tempo --- 39.928353786468506 seconds ---
brega
Tempo --- 35.15240550041199 seconds ---
Tempo total --- 204.73141384124756 seconds ---
